<a href="https://colab.research.google.com/github/EduardoGarrido90/tutorials_data_science/blob/main/California_MLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiple Linear Regression model of the California Housing Dataset

# MODEL SPECIFICATION AND ESTIMATION

The steps to estimate a MLR model wrt data are always the same ones.

1. First, we load the data. No data, no party!
2. We specify the model equation with respect to the variables of the data.
3. We fit the model and show the summary.

Then, we show two results:

1. The summary of the econometric model.

In [ ]:
from sklearn.datasets import fetch_california_housing
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

# Load California housing dataset
california_data = fetch_california_housing()
df_california = pd.DataFrame(california_data.data, columns=california_data.feature_names)
df_california['MedHouseVal'] = california_data.target  # Median house value

# Selecting a few predictors for the regression model
predictors_california = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
df_california_subset = df_california[predictors_california + ['MedHouseVal']]

# Linear regression model using statsmodels
model_california = smf.ols('MedHouseVal ~ MedInc + HouseAge + AveRooms + AveBedrms + Population + AveOccup + Latitude + Longitude', data=df_california_subset)
results = model_california.fit()

# Summary of the model
summary_california = results.summary()
summary_california


2. The equation of the model. Just modifying the dependent variable, you can modify this code to suit your particular needs.

In [ ]:
from IPython.display import Latex
# Corrected approach to generate the LaTeX representation of the model equation
dependent_var = 'MEDV'  # Name of the dependent variable
latex_equation = dependent_var + ' = '
for i, (param, value) in enumerate(zip(results.params.index, results.params.values)):

    if param == 'Intercept':
        latex_equation += f'{value:.4f}'
    else:
        sign = '+' if value >= 0 else '-'
        latex_equation += f' {sign} {abs(value):.4f} \\times {param}'

latex_equation = '$' + latex_equation + '$'
display(Latex(latex_equation))



# MODEL VALIDATION

**Checking multicolinearity**

Variance Inflation Factor (VIF): The VIF quantifies how much the variance of an estimated regression coefficient increases if your predictors are correlated. A VIF above 5-10 indicates high multicollinearity.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import scipy.stats as stats

# Calculate VIFs for each variable
X = df_california_subset[predictors_california]
X['Intercept'] = 1
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print(vif_data)

**Checking linearity and heterocedasticity**

A residual plot (residuals vs. fitted values) helps in checking the assumption of linearity and homoscedasticity (constant variance) of residuals.

A normality test, such as the Shapiro-Wilk test, can be used to assess the normality of the residuals. Additionally, a QQ plot (quantile-quantile plot) can visually assess the normality.

In [ ]:
# Residual plot
residuals = results.resid
fitted = results.fittedvalues

plt.figure(figsize=(10, 5))
plt.scatter(fitted, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values')
plt.show()

# Normality test and QQ plot
shapiro_test = stats.shapiro(residuals)
plt.figure(figsize=(5, 5))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('QQ Plot')
plt.show()

vif_data, shapiro_test

**Checking Autocorrelation**

Autocorrelation occurs when the residuals are not independent of each other. The Durbin-Watson test is commonly used to detect this.

A Durbin-Watson statistic close to 2 suggests no autocorrelation; values below 1 or above 3 are a cause for concern.

In [ ]:
# Durbin-Watson test for autocorrelation
durbin_watson_statistic = sm.stats.stattools.durbin_watson(residuals)
durbin_watson_statistic

**Checking Endogeneity**

Exogeneity implies that the predictors (independent variables) are not correlated with the error term. Violation of this assumption is known as endogeneity.

One common method to test for exogeneity is to look for correlation between the residuals and the independent variables, which can suggest endogeneity issues.

In [ ]:
# Cheking endogeneity
correlation_data = pd.DataFrame()
for predictor in predictors_california:
    correlation_data[predictor] = [df_california_subset[predictor].corr(residuals)]

correlation_data

**Checking specification**

Plotting Residuals vs. Predictors: This helps in visually inspecting whether the relationship between predictors and the response variable is linear or if there are any obvious patterns or non-linear trends in the residuals.

In [ ]:
from statsmodels.stats.diagnostic import linear_reset



# Plotting residuals vs. predictors for visual inspection
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(15, 10))
axs = axs.flatten()
for i, predictor in enumerate(predictors_california):
    axs[i].scatter(df_california_subset[predictor], residuals)
    axs[i].set_title(f'Residuals vs {predictor}')
    axs[i].set_xlabel(predictor)
    axs[i].set_ylabel('Residuals')

# Adjust layout and show plot
plt.tight_layout()
plt.show()

Ramsey's RESET Test: This test is specifically designed to detect omitted variable bias and incorrect functional form. It tests whether higher-order terms (like squared or cubic terms) of the fitted values help in explaining the response variable.

In [ ]:
# Ramsey's RESET test for model specification
reset_test = linear_reset(results, power=3, use_f=True)
reset_test

The RESET test results in an F-statistic of approximately 1098.23 with a very small p-value (almost 0).

This significant p-value indicates that there is evidence of omitted variables or that the model may have an incorrect functional form, such as missing non-linear components.